In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Caminho das pastas de treinamento
treinamento_dir = 'image/treinamento'
treinamento_boa_dir = os.path.join(treinamento_dir, 'boa')
treinamento_ruim_dir = os.path.join(treinamento_dir, 'ruim')

# Caminho para salvar o modelo
modelo_path = 'model/modelo_solda.h5'

In [ ]:
# Verifica se o modelo já existe, se existir, carrega o modelo
if os.path.exists(modelo_path):
    model = tf.keras.models.load_model(modelo_path)
    print("Modelo carregado, continuando o treinamento...")
else:
    # Caso contrário, cria e treina um novo modelo
    print("Treinando um novo modelo...")

    model = tf.keras.Sequential([
        # Camada convolucional 1
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # Camada convolucional 2
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        # Camada densa
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')  # Para classificação "boa" e "ruim"
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Gerador de imagens de treinamento
train_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    treinamento_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'  # 'sparse' pois temos 2 classes (boa, ruim)
)

In [ ]:
# Treinamento do modelo
model.fit(train_generator, epochs=10)  # Aqui você pode ajustar o número de épocas conforme necessário


In [ ]:
# Salvar o modelo treinado
model.save(modelo_path)
print("Modelo salvo em: ", modelo_path)